<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big%20Data/Big_Data_010_Trabalho_2_Sentimento_Noriaki_Suda_Wakabayashi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-tweets
*   Selecione a opção "Adicionar Atalho ao Drive"

In [1]:
!pip install mrjob nltk twython

     |████████████████████████████████| 439 kB 4.0 MB/s 


In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
!ls "/content/drive/My Drive/twitter_cs"

negative-words.txt  sample_clean_larger.csv	twcs_clean.csv
positive-words.txt  sample_clean_larger.csv.gz	twcs_clean.csv.gz
sample_clean.csv    sample.csv			twcs.csv


In [44]:
!cat "/content/drive/My Drive/twitter_cs/sample_clean.csv" | head

tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡,119236,
119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,"@105835 Your business means a lot to us. Please DM your name  zip code and additional details about your concern. ^RR https://t.co/znUu1VJn9r",,119239
119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm sure you won't! Because you don't have to!,119238,
119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,"@105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 option 1  4  3 (Leave a message) to request a call back",119241,119242
119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've tried leaving a voicemail several times in the past week https://t.co/NxVZj

# Ver porque nao esta saindo resultado pelo logger de warning

In [117]:
%%file trab2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import TextProtocol
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re
import logging

class Sentiment(MRJob):

  OUTPUT_PROTOCOL = TextProtocol  
  
  def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
            mapper=self.mapper,
            combiner=self.combiner,
            reducer=self.reducer),
            MRStep(reducer=self.reducer_order)
        ]

  def mapper_init(self):
    nltk.download('vader_lexicon', quiet=True)
    self.sentiment = SentimentIntensityAnalyzer()

  def mapper(self, _, line):
    try:

      tweet_id, author_id, inbound, created_at, text, response_tweet_id, in_response_to_tweet_id = line.split(",")

      author_id = author_id.lower().strip()
      flag_author = bool(re.match("^[0-9]+$", author_id))

      if inbound == "True":
        raise ValueError("Inbound") 

      elif author_id == "author_id":
        raise ValueError("Primeira Linha")

      elif flag_author == True:
        raise ValueError("Author")

      polaridade = self.sentiment.polarity_scores(text)['compound']

      yield author_id, (polaridade, 1)

    except ValueError as error:
      logging.warning(error.args)
      logging.warning("linha descartada: "+line)
      pass

  def combiner(self, key, values):
    try:
      polaridade_sum = 0
      quantidade = 0
      for value in values:
        polaridade_sum += value[0]
        quantidade += value[1]
    
      yield key, (polaridade_sum, quantidade)
    
    except:
      logging.warning("Deu ruim Combiner")     
      pass

  def reducer(self, key, values):
    try:
      polaridade_sum = 0
      quantidade = 0
      for value in values:
        polaridade_sum += value[0]
        quantidade += value[1]

      yield str(((polaridade_sum/quantidade)+1)*10000).zfill(6), key

    except:
      logging.warning("Deu ruim reducer")     
      pass
  
  def reducer_order(self, quantity, keys):
    try:
      for key in keys:
        yield key, str(quantity)
    except:
      logging.warning("Deu ruim reducer 2")

if __name__ == '__main__':
    logging.basicConfig(filename="log_trabalho2.txt",level=logging.INFO)
    Sentiment.run()

Overwriting trab2.py


In [118]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/sample_clean.csv" 

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/trab2.root.20211104.030348.237403
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/trab2.root.20211104.030348.237403/output
Streaming final output from /tmp/trab2.root.20211104.030348.237403/output...
virgintrains	11710.25
british_airways	12015.0
sprintcare	12263.0
upshelp	12263.0
applesupport	15644.153846153846
southwestair	16114.000000000002
hpsupport	16369.0
chasesupport	13182.0
tesco	14806.249999999998
spotifycares	15123.250000000002
o2	17783.0
ask_spectrum	7268.0
comcastcares	8783.0
Removing temp directory /tmp/trab2.root.20211104.030348.237403...


In [114]:
# Sanity Check pra ver se esta agrupando certo
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon', quiet=True)
analyzer = SentimentIntensityAnalyzer()

df = pd.read_csv("/content/drive/My Drive/twitter_cs/sample_clean.csv" )

df["test"] = df.author_id.str.match("^[0-9]+$")

df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]

df[df["test"]==False].filter(["author_id", "compound"]).groupby("author_id")["compound"].mean().add(1).sort_values()

author_id
Ask_Spectrum       0.726800
comcastcares       0.878300
VirginTrains       1.171025
British_Airways    1.201500
UPSHelp            1.226300
sprintcare         1.226300
ChaseSupport       1.318200
Tesco              1.480625
SpotifyCares       1.512325
AppleSupport       1.564415
SouthwestAir       1.611400
HPSupport          1.636900
O2                 1.778300
Name: compound, dtype: float64

In [ ]:
!cat log_trabalho2.txt

In [58]:
!ls

drive  log_trabalho2.txt  log.txt  sample_data	trab2.py


In [52]:
!rm log_trabalho2.txt

In [50]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/sample_clean_larger.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/trab2.root.20211104.013927.970603
Running step 1 of 1...
job output is in /tmp/trab2.root.20211104.013927.970603/output
Streaming final output from /tmp/trab2.root.20211104.013927.970603/output...
adobecare	0.33681000000000005
airasiasupport	0.40325428571428557
airbnbhelp	0.5175847826086956
airtel_care	0.5359299065420557
alaskaair	0.28451428571428566
aldiuk	0.364477027027027
amazonhelp	0.2028177449168204
americanair	0.37496927083333315
applesupport	0.43845113533151786
arbyscares	0.38129047619047624
argoshelpers	0.20670472440944876
ask_progressive	0.39189999999999997
ask_spectrum	0.42600185873605906
ask_wellsfargo	0.5134346666666666
askamex	0.3208982300884956
askciti	0.6313333333333334
askdsc	0.26553333333333334
askebay	0.32142747252747256
asklyft	0.31073366336633657
askpanera	0.4997384615384616
askpapajohns	0.24429019607843133
askpaypal	0.41390217391304357
askplaystat

In [66]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/twcs_clean.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/trab2.root.20211104.015258.012568
Running step 1 of 1...
tcmalloc: large alloc 2067750912 bytes == 0x560383576000 @  0x7fc5e1e3e1e7 0x560381e67718 0x560381e665a1 0x7fc5e181cbf7 0x560381e6702a
job output is in /tmp/trab2.root.20211104.015258.012568/output
Streaming final output from /tmp/trab2.root.20211104.015258.012568/output...
adobecare	0.3738690632247648
airasiasupport	0.35531852833424754
airbnbhelp	0.4719709971830961
airtel_care	0.5125115271733871
alaskaair	0.3277007221182133
aldiuk	0.4461229356667411
amazonhelp	0.20070675989171965
americanair	0.366910825807849
applesupport	0.43707453303385935
arbyscares	0.3454461134453776
argoshelpers	0.1905811894200779
ask_progressive	0.4415361111111109
ask_spectrum	0.4077915081206631
ask_wellsfargo	0.5098603457376647
askamex	0.3340277073428465
askciti	0.5837064935064907
askdsc	0.17663952380952377
askebay	0.3491372083419353
ask

In [41]:
drive.flush_and_unmount()